<a href="https://colab.research.google.com/github/BFvandy/Nissan-Text-Mining-POC-with-Transformer/blob/main/SW_QA_prefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Prep

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
cur_path = "/content/drive/My Drive/Nissan Internship/SW 2020-2022 Verbatim Small SUV"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/Nissan Internship/SW 2020-2022 Verbatim Small SUV


# Load Transformer

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.2 MB/s 
     |████████████████████████████████| 101 kB 12.8 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import pipeline

In [ ]:
model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

In [ ]:
res

{'answer': 'gives freedom to the user',
 'end': 84,
 'score': 0.2117149531841278,
 'start': 59}

# Load Dataset

In [ ]:
comments = pd.read_csv('all_SW_cmt_raw.csv', encoding_errors='ignore')
comments = comments.rename(columns={'Verbatim Text': 'Text'})

In [ ]:
comments

,Make,Model,Purchased,Study Year,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text
0,Honda,CR-V,Honda CR-V Special Edition FWD,2022,2021,Honda Civic,2019.0,Honda HR-V,2022.0,Sentiment Not Calculated,Age 41,Female,White,Income Unknown,The engine start stop/ gas save mode is not ve...
1,Nissan,Rogue,Nissan Rogue SV AWD,2022,2021,Ford Fusion,2012.0,Toyota Highlander,2021.0,Sentiment Not Calculated,Age 35,Male,White,"$100,001 - $125,000",We were not told the price of the vehicle goin...
2,Nissan,Rogue,Nissan Rogue S AWD,2022,2021,Chevrolet Spark,2016.0,Hyundai Santa Fe,2021.0,Sentiment Not Calculated,Age 57,Male,White,Income Unknown,My Nissan Rogue is the perfect balance between...
3,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,Nissan Versa Note (Hatchback),2018.0,Lexus LX,2022.0,Sentiment Not Calculated,Age 49,Female,Asian,"$65,001 - $70,000",I wish :\n1.there is a CD player installed. T...
4,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,(None),NaN,Honda Civic,2015.0,Sentiment Not Calculated,Age 54,Female,White,"$200,001 - $300,000","The 2022 Honda CRV is a nice looking car, with..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106999,Honda,CR-V,Honda CR-V EX/EX-L FWD,2020,2020,None,18.0,None,16.0,Sentiment Not Calculated,Age 34,Female,Hispanic,"$150,001 - $175,000",G
107000,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,Age 42,Male,White,"$100,001 - $125,000",.
107001,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,Age 42,Male,White,"$100,001 - $125,000",.
107002,Honda,CR-V,Honda CR-V LX FWD,2020,2019,Honda CR-V,2015.0,Honda HR-V,2019.0,Sentiment Not Calculated,Age 59,Male,White,"$50,001 - $55,000",I


In [ ]:
fun_labels = ['Dependable', 'Lasts long', 'Value for money', 'Quality fit and finish', 'Attractive styling'
                                    , 'Safe', 'Retains resale value', 'Driver comfort', 'Fun to drive', 
                                    'Advanced features', 'Responsive handling', 'Prestigious', 'Dealerships', 
        'Fuel efficient', 'Quick acceleration', 'Environmentally friendly', 'Affordable']
pers_labels = ['Trusted', 'Leader', 'Responsible', 'Confident', 'Innovative', 'Exciting', 'Practical', 'Adventurous', 
               'Passionate', 'Distinctive', 'Youthful', 'Aggressive']

all_labels = fun_labels+pers_labels
all_labels_lower = [i.lower() for i in all_labels]

In [ ]:
comments[all_labels_lower] = pd.NaT
comments

,Make,Model,Purchased,Study Year,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Honda,CR-V,Honda CR-V Special Edition FWD,2022,2021,Honda Civic,2019.0,Honda HR-V,2022.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Nissan,Rogue,Nissan Rogue SV AWD,2022,2021,Ford Fusion,2012.0,Toyota Highlander,2021.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Nissan,Rogue,Nissan Rogue S AWD,2022,2021,Chevrolet Spark,2016.0,Hyundai Santa Fe,2021.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,Nissan Versa Note (Hatchback),2018.0,Lexus LX,2022.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,(None),NaN,Honda Civic,2015.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106999,Honda,CR-V,Honda CR-V EX/EX-L FWD,2020,2020,None,18.0,None,16.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
107000,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
107001,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
107002,Honda,CR-V,Honda CR-V LX FWD,2020,2019,Honda CR-V,2015.0,Honda HR-V,2019.0,Sentiment Not Calculated,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


# Implementation

In [ ]:
import time

start = time.time()

for i in range(len(comments)):
  for j in all_labels_lower:
    try:
      QA_input = {
          'question': 'Is ' + j + ' mentioned?',
          'context': comments.at[i, 'Text']
      }
      res = nlp(QA_input)
      comments.at[i, j] = 1 if res['score'] > 0.1 else 0 
    except:
      pass
  if i % 2000 == 0:
    print(i)
  else:
    pass
    
end = time.time()
print((end - start)/60)


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
391.3201100349426


In [ ]:
comments

,Make,Model,Purchased,Study Year,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,...,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Honda,CR-V,Honda CR-V Special Edition FWD,2022,2021,Honda Civic,2019.0,Honda HR-V,2022.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
1,Nissan,Rogue,Nissan Rogue SV AWD,2022,2021,Ford Fusion,2012.0,Toyota Highlander,2021.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
2,Nissan,Rogue,Nissan Rogue S AWD,2022,2021,Chevrolet Spark,2016.0,Hyundai Santa Fe,2021.0,Sentiment Not Calculated,...,0,0,0,1,0,0,0,0,0,0
3,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,Nissan Versa Note (Hatchback),2018.0,Lexus LX,2022.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
4,Honda,CR-V,Honda CR-V EX/EX-L FWD,2022,2022,(None),NaN,Honda Civic,2015.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106999,Honda,CR-V,Honda CR-V EX/EX-L FWD,2020,2020,None,18.0,None,16.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
107000,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
107001,Honda,CR-V,Honda CR-V EX/EX-L AWD,2020,2019,GMC Terrain,2013.0,Honda CR-V,2020.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0
107002,Honda,CR-V,Honda CR-V LX FWD,2020,2019,Honda CR-V,2015.0,Honda HR-V,2019.0,Sentiment Not Calculated,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
comments.to_csv('SW_cmt_prefiltered.csv', index=False)
print("Save it")

Save it


In [ ]:
for i in range(len(all_labels_lower)):
  label = all_labels_lower[i]
  count = (comments[label] == 1).sum()
  print(label)
  print(count)

dependable
16745
lasts long
7218
value for money
4894
quality fit and finish
9237
attractive styling
7306
safe
9473
retains resale value
2548
driver comfort
10760
fun to drive
8709
advanced features
10703
responsive handling
7821
prestigious
10613
dealerships
2502
fuel efficient
5813
quick acceleration
729
environmentally friendly
1204
affordable
4315
trusted
11024
leader
764
responsible
7173
confident
7359
innovative
3868
exciting
5212
practical
9014
adventurous
4708
passionate
6607
distinctive
10672
youthful
1872
aggressive
5409
